In [1]:
import pandas as pd
import spacy
import os
import spacy
import polars as pl
import mlflow
import mlflow.spacy
from pathlib import Path
import json
from spacy.util import minibatch, compounding
from spacy.cli.train import train
from dotenv import load_dotenv
import sys
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../..'))
from app.data_handling.preprocessing import process_ingredient, transform_ingredients_to_tokens

# Create Dataframe and save as parquet

In [2]:
load_dotenv()

True

In [3]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL"))

In [4]:
model_uri = "models:/recipe_NER@prod"
ner = mlflow.spacy.load_model(model_uri=model_uri)

/home/naabzer/develop/recipe_tm/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
nlg_ds = pd.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv', converters={'ingredients': pd.eval}, nrows=10)

In [6]:
nlg_ds = pl.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv',  n_rows=10)
nlg_ds = nlg_ds.with_columns(pl.col("ingredients").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.col("directions").str.json_decode())

In [7]:
nlg_ds

,title,ingredients,directions,link,source,NER
i64,str,list[str],list[str],str,str,str
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…"
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…"
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…"
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…"
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…"
5,"""Cheeseburger Potato Soup""","[""6 baking potatoes"", ""1 lb. of extra lean ground beef"", … ""1 (8 oz.) carton sour cream (optional)""]","[""Wash potatoes; prick several times with a fork."", ""Microwave them with a wet paper towel covering the potatoes on high for 6-8 minutes."", … ""Sprinkle with remaining cheese, bacon and green onions.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""baking potatoes"", ""extra lea…"
6,"""Rhubarb Coffee Cake""","[""1 1/2 c. sugar"", ""1/2 c. butter"", … ""1 tsp. vanilla""]","[""Cream sugar and butter."", ""Add egg and beat well."", … ""Pour into greased 9 x 13-inch pan and add Topping.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""butter"", ""egg"", ""bu…"
7,"""Scalloped Corn""","[""1 can cream-style corn"", ""1 can whole kernel corn"", … ""pepper to taste""]","[""Mix together both cans of corn, crackers, egg, 2 teaspoons of melted butter and pepper and place in a buttered baking dish."", ""Dot with remaining 4 teaspoons of butter."", ""Bake at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cream-style corn"", ""whole ke…"
8,"""Nolan'S Pepper Steak""","[""1 1/2 lb. round steak (1-inch thick), cut into strips"", ""1 can drained tomatoes, cut up (save liquid)"", … ""1/4 c. oil""]","[""Roll steak strips in flour."", ""Brown in skillet."", … ""Serve over hot cooked rice.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""tomatoes"", ""water"", ""onions""…"


In [8]:
nlg_ds['ingredients'][0]


""
str
"""1 c. firmly packed brown sugar"""
"""1/2 c. evaporated milk"""
"""1/2 tsp. vanilla"""
"""1/2 c. broken nuts (pecans)"""
"""2 Tbsp. butter or margarine"""
"""3 1/2 c. bite size shredded ri…"


In [95]:
def transform_ds_to_BOW(ds: pl.dataframe.frame.DataFrame, create_vocab=False):  # Assume recipeNLG dataset, where each ingredient line is a list item
    tokens = []
    preps = []
    optionals = []
    for i, ingredients in enumerate(ds['ingredients']):
        datapoint_obj = transform_ingredients_to_tokens(ingredients, ner, create_vocab)
        tokens.append(datapoint_obj['foods'])
        preps.append(json.dumps(datapoint_obj['preps']))
        optionals.append(datapoint_obj['optionals'])
    new_ds = ds.with_columns(
        pl.Series(name='BOW_tokens', values=tokens),
        pl.Series(name='preps', values=preps),
        pl.Series(name='optionals', values=optionals)
    )
    return new_ds

In [96]:
nlg_ds

,title,ingredients,directions,link,source,NER,BOW_tokens,preps,optionals
i64,str,list[str],list[str],str,str,str,list[str],struct[0],list[bool]
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…","[""brown_sugar"", ""milk"", … ""butter""]",{},"[false, false, … false]"
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…","[""beef"", ""chicken_breast"", … ""sour_cream""]",{},"[false, false, … false]"
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…","[""corn"", ""cream_cheese"", … ""pepper""]",{},"[false, false, … false]"
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…","[""chicken"", ""chicken_gravy"", ""cream_of_mushroom_soup""]",{},"[false, false, false]"
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…","[""peanut_butter"", ""graham_cracker_crumb"", … ""chocolate_chip""]",{},"[false, false, … false]"
5,"""Cheeseburger Potato Soup""","[""6 baking potatoes"", ""1 lb. of extra lean ground beef"", … ""1 (8 oz.) carton sour cream (optional)""]","[""Wash potatoes; prick several times with a fork."", ""Microwave them with a wet paper towel covering the potatoes on high for 6-8 minutes."", … ""Sprinkle with remaining cheese, bacon and green onions.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""baking potatoes"", ""extra lea…","[""bake_potato"", ""beef"", … ""sour_cream""]",{},"[false, false, … true]"
6,"""Rhubarb Coffee Cake""","[""1 1/2 c. sugar"", ""1/2 c. butter"", … ""1 tsp. vanilla""]","[""Cream sugar and butter."", ""Add egg and beat well."", … ""Pour into greased 9 x 13-inch pan and add Topping.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""butter"", ""egg"", ""bu…","[""sugar"", ""butter"", … ""rhubarb""]",{},"[false, false, … false]"
7,"""Scalloped Corn""","[""1 can cream-style corn"", ""1 can whole kernel corn"", … ""pepper to taste""]","[""Mix together both cans of corn, crackers, egg, 2 teaspoons of melted butter and pepper and place in a buttered baking dish."", ""Dot with remaining 4 teaspoons of butter."", ""Bake at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cream-style corn"", ""whole ke…","[""cream_-_style"", ""corn"", … ""pe

In [97]:
nlg_ds = transform_ds_to_BOW(nlg_ds, create_vocab=True)

In [98]:
nlg_ds

,title,ingredients,directions,link,source,NER,BOW_tokens,preps,optionals
i64,str,list[str],list[str],str,str,str,list[str],str,list[bool]
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…","[""brown_sugar"", ""milk"", … ""butter""]","""{}""","[false, false, … false]"
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…","[""beef"", ""chicken_breast"", … ""sour_cream""]","""{""beef"": [""chip"", ""cut"", ""up""]…","[false, false, … false]"
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…","[""corn"", ""cream_cheese"", … ""pepper""]","""{""cream_cheese"": [""cube""], ""bu…","[false, false, … false]"
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…","[""chicken"", ""chicken_gravy"", ""cream_of_mushroom_soup""]","""{}""","[false, false, false]"
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…","[""peanut_butter"", ""graham_cracker_crumb"", … ""chocolate_chip""]","""{}""","[false, false, … false]"
5,"""Cheeseburger Potato Soup""","[""6 baking potatoes"", ""1 lb. of extra lean ground beef"", … ""1 (8 oz.) carton sour cream (optional)""]","[""Wash potatoes; prick several times with a fork."", ""Microwave them with a wet paper towel covering the potatoes on high for 6-8 minutes."", … ""Sprinkle with remaining cheese, bacon and green onions.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""baking potatoes"", ""extra lea…","[""bake_potato"", ""beef"", … ""sour_cream""]","""{""beef"": [""ground""], ""cheddar_…","[false, false, … true]"
6,"""Rhubarb Coffee Cake""","[""1 1/2 c. sugar"", ""1/2 c. butter"", … ""1 tsp. vanilla""]","[""Cream sugar and butter."", ""Add egg and beat well."", … ""Pour into greased 9 x 13-inch pan and add Topping.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""butter"", ""egg"", ""bu…","[""sugar"", ""butter"", … ""rhubarb""]","""{""rhubarb"": [""finely"", ""cut""]}""","[false, false, … false]"
7,"""Scalloped Corn""","[""1 can cream-style corn"", ""1 can whole kernel corn"", … ""pepper to taste""]","[""Mix together both cans of corn, crackers, egg, 2 teaspoons of melted butter and pepper and place in a buttered baking dish."", ""Dot with remaining 4 teaspoons of 

In [99]:
nlg_ds['ingredients'][2]

""
str
"""2 (16 oz.) pkg. frozen corn"""
"""1 (8 oz.) pkg. cream cheese, c…"
"""1/3 c. butter, cubed"""
"""1/2 tsp. garlic powder"""
"""1/2 tsp. salt"""
"""1/4 tsp. pepper"""


In [100]:
nlg_ds['BOW_tokens'][2]

""
str
"""corn"""
"""cream_cheese"""
"""butter"""
"""garlic_powder"""
"""salt"""
"""pepper"""


In [101]:
nlg_ds['preps'][2]

'{"cream_cheese": ["cube"], "butter": ["cube"]}'

In [102]:
nlg_ds.write_parquet('test.parquet')

# Test some queries using DuckDB and parquet

In [103]:
import duckdb

In [104]:
duckdb.sql("SELECT * FROM 'test.parquet' AS tbl WHERE list_contains(tbl.BOW_tokens, 'egg')").pl()

C0,title,ingredients,directions,link,source,NER,BOW_tokens,preps,optionals
i64,str,list[str],list[str],str,str,str,list[str],str,list[bool]
6,"""Rhubarb Coffee Cake""","[""1 1/2 c. sugar"", ""1/2 c. butter"", … ""1 tsp. vanilla""]","[""Cream sugar and butter."", ""Add egg and beat well."", … ""Pour into greased 9 x 13-inch pan and add Topping.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""butter"", ""egg"", ""bu…","[""sugar"", ""butter"", … ""rhubarb""]","""{""rhubarb"": [""finely"", ""cut""]}""","[false, false, … false]"
7,"""Scalloped Corn""","[""1 can cream-style corn"", ""1 can whole kernel corn"", … ""pepper to taste""]","[""Mix together both cans of corn, crackers, egg, 2 teaspoons of melted butter and pepper and place in a buttered baking dish."", ""Dot with remaining 4 teaspoons of butter."", ""Bake at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cream-style corn"", ""whole ke…","[""cream_-_style"", ""corn"", … ""pepper""]","""{""saltine_cracker"": [""crush""],…","[false, false, … false]"


In [106]:
duckdb.sql("COPY(SELECT preps::JSON AS preps, * EXCLUDE (preps) FROM 'test.parquet') TO 'test.parquet' (FORMAT PARQUET, OVERWRITE TRUE)")

In [131]:
duckdb.sql("SELECT json_extract(preps, ['butter', 'rhubarb']) FROM 'test.parquet'")

┌───────────────────────────────────────────────────────────┐
│ json_extract(preps, main.list_value('butter', 'rhubarb')) │
│                          json[]                           │
├───────────────────────────────────────────────────────────┤
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ ['["cube"]', NULL]                                        │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ [NULL, '["finely","cut"]']                                │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
├───────────────────────────────────────────────────────────┤
│                          10 rows                          │
└───────

In [133]:
duckdb.sql("SELECT json_exists(preps, ['butter', 'rhubarb']) FROM 'test.parquet'")

┌──────────────────────────────────────────────────────────┐
│ json_exists(preps, main.list_value('butter', 'rhubarb')) │
│                        boolean[]                         │
├──────────────────────────────────────────────────────────┤
│ [false, false]                                           │
│ [false, false]                                           │
│ [true, false]                                            │
│ [false, false]                                           │
│ [false, false]                                           │
│ [false, false]                                           │
│ [false, true]                                            │
│ [false, false]                                           │
│ [false, false]                                           │
│ [false, false]                                           │
├──────────────────────────────────────────────────────────┤
│                         10 rows                          │
└───────────────────────

In [271]:
ingredients = ['butter']
preps = {'butter': ['cube'], 'cream_cheese': ['cube']}

In [272]:
list(preps.keys())

['butter', 'cream_cheese']

In [275]:
def construct_ingredient_query(pq_path: str, ingredients: list, preps: dict):
    prep_keys = list(preps.keys())
    base_sql = (f"SELECT *, json_extract(preps, {prep_keys}) AS prep_lists, from_json(prep_lists->>{0}, '[\"VARCHAR\"]') as testing"
                f" FROM '{pq_path}'"
                " AS tbl"
                f" WHERE list_has_all(tbl.BOW_tokens, {ingredients})"
               )
    prep_filter = ""
    for i, (key, value) in enumerate(preps.items()):
        prep_filter += (f" AND (prep_lists->>{i}) IS NOT NULL"
                        f" AND list_has_all(from_json(prep_lists->>{i}, '[\"VARCHAR\"]'), {value})"
                       )
    sql = base_sql + prep_filter
    return sql

In [276]:
duckdb.sql(construct_ingredient_query('test.parquet', ingredients, preps)).pl()

preps,C0,title,ingredients,directions,link,source,NER,BOW_tokens,optionals,prep_lists,testing
str,i64,str,list[str],list[str],str,str,str,list[str],list[bool],list[str],list[str]
"""{""cream_cheese"": [""cube""], ""bu…",2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…","[""corn"", ""cream_cheese"", … ""pepper""]","[false, false, … false]","[""[""cube""]"", ""[""cube""]""]","[""cube""]"
